### Excel 自動輸出樞紐分析表

內容主要參考自[pandas.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)<br>
following content first written in 2021.Jun.<br>

 - 說明:
1. 還是要先透過excel檔案先行透過自己所欲的方式去排雜
2. "主文/回文"的欄位還是要先透過excel處理，把"回文xx"全部更動為"回文"

#### 前置作業

In [ ]:
#設定環境
#-*-coding:UTF-8-*- 

import os, time, glob, socket

print("【日期時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【工作目錄】{}".format(os.getcwd()))
print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.gethostname())))

#設定自動存檔時間

%autosave 60

In [ ]:
#安裝pandas套件

import pandas as pd

In [ ]:
#匯入excel檔
##要自行更換路徑

import warnings

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df = pd.read_excel(r"C:\Users\mingyanlin\OneDrive\文件\Task\eLand\【Unilever】火鍋專案分析報告\火鍋報告_促銷優惠.xlsx", 
                       engine="openpyxl", 
                       index_col='監測主題')

#### 1、資料切割與清整

In [ ]:
#將 "來源網站" 的內容，切割、另建立成 "網站" 、 "頻道" 2個不同的欄位
df[['網站', '頻道']] = df.來源網站.str.split(' > ', expand = True)

In [ ]:
#將 "發佈時間" 的內容，切割、另建立成 "日期" 、 "時間" 2個不同的欄位
df[['日期', '時間']] = df.發佈時間.str.split(' ', expand = True)

In [ ]:
#將 "發佈時間" 的內容，切割、另建立 "年份" 、 "月" 、 "日" 3個不同的欄位
df[['年份', '月', '日']] = df.日期.str.split('/', expand = True)

In [ ]:
#將 "時間" 的內容，切割、另建立 "時" 、 "分" 、 "秒" 3個不同的欄位
df[['時', '分', '秒']] = df.時間.str.split(':', expand = True)

In [ ]:
#將所有的開頭為回文的字串，都取代成 "回文" 2字
df = df.replace({"主文/回文" : r"^回文.*$"}, {"主文/回文" : "回文"}, regex= True)

In [ ]:
#檢視資料目前形式，df.head(顯示前幾列的資料)
df.head(3)

In [ ]:
ds = df["網站"] != "Facebook公開社團"

with pd.ExcelWriter(r"C:\Users\mingyanlin\OneDrive\文件\Task\eLand\【Unilever】火鍋專案分析報告\促銷優惠_python.xlsx", 
                    engine='xlsxwriter',
                    engine_kwargs={'options': {'strings_to_numbers': True, 'strings_to_urls' : False}}) as writer:
    df[ds].to_excel(writer, sheet_name='文章列表')

#### 2、樞紐分析

In [ ]:
#情緒標記 x 主回文
pt1 = pd.pivot_table(df, values='來源', 
               columns="情緒標記", 
               index="主文/回文", 
               aggfunc='count', fill_value=0)

In [ ]:
#情緒標記 x 來源網站
pt2 = pd.pivot_table(df, values='來源', 
               columns="情緒標記", 
               index="來源網站", 
               aggfunc='count', fill_value=0)

In [ ]:
#來源網站 x 主/回文
pt3 = pd.pivot_table(df, values='來源', 
               columns="主文/回文", 
               index="來源網站", 
               aggfunc='count', fill_value=0)

In [ ]:
#by月 x 情緒標記
pt4 = pd.pivot_table(df, values='來源',
               index='月', 
               columns='情緒標記', 
               aggfunc='count', fill_value=0)

In [ ]:
#by月 x 主/回文
pt5 = pd.pivot_table(df, values='來源', 
               index='月', 
               columns='主文/回文', 
               aggfunc='count', fill_value=0)

In [ ]:
#來源網站 x 情緒標記 x 主回文
pt6 = pd.merge(pt2, pt3, on='來源網站')

In [ ]:
#by月 x 情緒標記 x 主回文
pt7 = pd.merge(pt4, pt5, on='月')

In [ ]:
#by月 x 每小時聲量數
pt8 = pd.pivot_table(df, values='來源', 
               columns="月", 
               index="時", 
               aggfunc='count', fill_value=0)

In [ ]:
#來源 x 主回文
pt9 = pd.pivot_table(df, values='來源網站', 
               columns="主文/回文", 
               index="來源", 
               aggfunc='count', fill_value=0)

In [ ]:
#來源 x 情緒標記
pt10 = pd.pivot_table(df, values='來源網站', 
               columns="情緒標記", 
               index="來源", 
               aggfunc='count', fill_value=0)

In [ ]:
#來源 x 情緒標記 x 主回文
pt11 = pd.merge(pt9, pt10, on='來源')

#### 3、匯出資料成Excel檔

1. 把製作完成的樞紐分析表匯出成excel檔
2. 要自行更換路徑跟檔名

In [ ]:
with pd.ExcelWriter(r"C:\Users\xphoe\樞紐_3人.xlsx", options={'strings_to_urls': False}) as writer:
    df.to_excel(writer, sheet_name='Data Set')
    pt1.to_excel(writer, sheet_name='情緒標記 x 主回文')
    pt2.to_excel(writer, sheet_name='來源網站 x 情緒標記')
    pt3.to_excel(writer, sheet_name='來源網站 x 主回文')
    pt4.to_excel(writer, sheet_name='by月 x 情緒標記')
    pt5.to_excel(writer, sheet_name='by月 x 主回文')
    pt6.to_excel(writer, sheet_name='來源網站 x 情緒標記 x 主回文')
    pt7.to_excel(writer, sheet_name='by月 x 情緒標記 x 主回文')
    pt8.to_excel(writer, sheet_name='by月 x 每小時聲量數')
    pt9.to_excel(writer, sheet_name='來源 x 主回文')
    pt10.to_excel(writer, sheet_name='來源 x 情緒標記')
    pt11.to_excel(writer, sheet_name='來源 x 情緒標記 x 主回文')